In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Head_and_Neck_Cancer"
cohort = "GSE151181"

# Input paths
in_trait_dir = "../../input/GEO/Head_and_Neck_Cancer"
in_cohort_dir = "../../input/GEO/Head_and_Neck_Cancer/GSE151181"

# Output paths
out_data_file = "../../output/preprocess/Head_and_Neck_Cancer/GSE151181.csv"
out_gene_data_file = "../../output/preprocess/Head_and_Neck_Cancer/gene_data/GSE151181.csv"
out_clinical_data_file = "../../output/preprocess/Head_and_Neck_Cancer/clinical_data/GSE151181.csv"
json_path = "../../output/preprocess/Head_and_Neck_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene and miRNA expression in radioiodine refractory and avid papillary thyroid carcinomas"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['histological variant: Classical', 'histological variant: Follicular', 'histological variant: NA', 'histological variant: non-neoplastic thyroid'], 1: ['tissue type: Primary tumor', 'tissue type: synchronous lymph node metastasis', 'tissue type: lymph node metastasis post RAI', 'tissue type: lymph node metastasis_2 post RAI', 'tissue type: lymph node metastasis_1 post RAI', 'tissue type: non-neoplastic thyroid'], 2: ['collection before/after rai: Before', 'collection before/after rai: After'], 3: ['patient id: pt_1', 'patient id: pt_2', 'patient id: pt_3', 'patient id: pt_5', 'patient id: pt_7', 'patient id: pt_8', 'patient id: pt_11', 'patient id: pt_12', 'patient id: pt_13', 'patient id: p

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this appears to be a SuperSeries containing gene expression data
# The title mentions "Gene and miRNA expression" so it likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait (Head and Neck Cancer):
# Based on the provided information, this appears to be a dataset about thyroid cancer
# Key 4 contains information about "patient rai response" which can be used as our trait
trait_row = 4

# For age:
# There is no age information in the sample characteristics
age_row = None

# For gender:
# There is no gender information in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert trait value (radioiodine response) to binary format"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: Refractory (resistant to treatment) = 1, Avid (responsive) = 0
    if 'refractory' in value.lower():
        return 1  # Refractory - disease is persistent/resistant
    elif 'avid' in value.lower():
        return 0  # Avid - disease responds to radioiodine
    else:
        return None

def convert_age(value):
    """Convert age value to numeric"""
    # No age data available
    return None

def convert_gender(value):
    """Convert gender value to binary format"""
    # No gender data available
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since we don't have direct access to the actual clinical data file,
# we'll note that trait information is available and has been recorded in the json file
if trait_row is not None:
    print(f"Clinical data is available for {cohort}.")
    print(f"Trait row identified: {trait_row} (patient rai response)")
    print(f"This information has been recorded in {json_path}")
    print("Note: Actual clinical data extraction would require the proper clinical_data.csv file.")


Clinical data is available for GSE151181.
Trait row identified: 4 (patient rai response)
This information has been recorded in ../../output/preprocess/Head_and_Neck_Cancer/cohort_info.json
Note: Actual clinical data extraction would require the proper clinical_data.csv file.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Head_and_Neck_Cancer/GSE151181/GSE151181-GPL21575_series_matrix.txt.gz


Gene data shape: (62976, 47)
First 20 gene/probe identifiers:
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the identifiers in the gene expression data

# The identifiers in the gene expression data are numeric strings (e.g., '23064070', '23064071')
# These appear to be probe IDs from a microarray platform (GPL23159) rather than standard gene symbols
# Standard human gene symbols would typically be letters or combinations of letters and numbers like "TP53", "BRCA1", etc.
# These numeric IDs will need to be mapped to standard gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'COL', 'ROW', 'SPOT_ID', 'CONTROL_TYPE', 'miRNA_ID', 'GENE_SYMBOL', 'GENE_NAME', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION']
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': ['328', '326', '324', '322', '320'], 'SPOT_ID': ['miRNABrightCorner30', 'Blank', 'Blank', 'Blank', 'Blank'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'miRNA_ID': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan]}

Searching for platform information in SOFT file:
Platform ID not found in first 100 lines

Searching for gene symbol information in SOFT file:
Found references to gene symbols:
#GENE_SYMBOL = Gene Symbol
ID	COL	ROW	SPOT_ID	CONTROL_TYPE	miRNA_ID	GENE_SYMBOL	GENE_NAME	ACCESSION_STRING	CHROMOSOMAL_LOCATION

Checking for additional annotation files 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Reassess the matrix files and identify the correct platform
matrix_files = [f for f in os.listdir(in_cohort_dir) if 'matrix' in f.lower()]
print(f"Available matrix files: {matrix_files}")

# From previous output we see two platforms:
# - GPL23159 (Agilent-070156 Human_miRNA_V21.0) - this is a miRNA array
# - GPL21575 (Clariom_S_Human) - this is a gene expression array

# We want gene expression data for our Head and Neck Cancer trait analysis
# Let's check if we have a Clariom array matrix file which would contain gene expression
clariom_matrix = [f for f in matrix_files if 'GPL21575' in f]
if clariom_matrix:
    print(f"Found Clariom gene expression array: {clariom_matrix[0]}")
    gene_matrix_path = os.path.join(in_cohort_dir, clariom_matrix[0])
    
    # Extract gene expression data from the correct matrix file
    gene_data = get_genetic_data(gene_matrix_path)
    print(f"Gene expression data shape from Clariom array: {gene_data.shape}")
    print(f"First few probe IDs: {gene_data.index[:5].tolist()}")
    
    # Get the annotation for this specific platform
    with gzip.open(soft_file, 'rt') as f:
        platform_section = False
        platform_annotation_text = ""
        for line in f:
            if line.startswith('!Platform_table_begin') and 'GPL21575' in line:
                platform_section = True
                continue
            elif line.startswith('!Platform_table_end') and platform_section:
                break
            elif platform_section:
                platform_annotation_text += line
    
    # Check if we found platform-specific annotation
    if platform_annotation_text:
        print("Found platform-specific annotation for Clariom array")
        # Parse the annotation to create mapping
        platform_annotation_df = pd.read_csv(io.StringIO(platform_annotation_text), sep='\t')
        print(f"Annotation columns: {platform_annotation_df.columns.tolist()}")
        
        # Check for gene symbol column
        gene_symbol_cols = [col for col in platform_annotation_df.columns if 'symbol' in col.lower()]
        if gene_symbol_cols:
            # Create mapping dataframe using the ID column and gene symbol column
            prob_col = platform_annotation_df.columns[0]  # First column is typically the probe ID
            gene_col = gene_symbol_cols[0]
            
            print(f"Using mapping from {prob_col} to {gene_col}")
            gene_mapping = platform_annotation_df[[prob_col, gene_col]].dropna(subset=[gene_col])
            gene_mapping = gene_mapping.rename(columns={prob_col: 'ID', gene_col: 'Gene'})
            gene_mapping['ID'] = gene_mapping['ID'].astype(str)
            
            # Apply gene mapping
            print(f"Gene mapping shape: {gene_mapping.shape}")
            mapped_gene_data = apply_gene_mapping(gene_data, gene_mapping)
            gene_data = mapped_gene_data
        else:
            print("No gene symbol column found in annotation")
    else:
        print("Platform-specific annotation not found in SOFT file")

# If mapping isn't possible or resulting data is empty, continue with original probe IDs
if 'gene_data' not in locals() or gene_data.shape[0] == 0:
    print("Direct mapping not successful. Using original probe IDs for downstream analysis.")
    # Continue using the original gene expression data
    # No need to reassign gene_data as it still contains the probe-level data
else:
    # If mapping was successful, apply normalization to handle gene synonyms
    print(f"Successfully mapped probes to genes. Shape: {gene_data.shape}")
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Final gene expression data shape after normalization: {gene_data.shape}")
    print(f"Sample of gene symbols: {gene_data.index[:10].tolist()}")

# Print final status
print(f"Final gene data shape: {gene_data.shape}")
print(f"Will proceed with {'gene-level' if gene_data.shape[0] > 0 else 'probe-level'} data for further analysis")


Available matrix files: ['GSE151181-GPL21575_series_matrix.txt.gz', 'GSE151181-GPL23159_series_matrix.txt.gz']
Found Clariom gene expression array: GSE151181-GPL21575_series_matrix.txt.gz


Gene expression data shape from Clariom array: (62976, 47)
First few probe IDs: ['1', '2', '3', '4', '5']


Platform-specific annotation not found in SOFT file
Successfully mapped probes to genes. Shape: (62976, 47)
Final gene expression data shape after normalization: (0, 47)
Sample of gene symbols: []
Final gene data shape: (0, 47)
Will proceed with probe-level data for further analysis


### Step 7: Data Normalization and Linking

In [8]:
# 1. Since we have empty gene expression data, mark dataset as unusable
print(f"Gene data shape: {gene_data.shape}")
print("Gene data is empty (0 rows). Cannot proceed with proper analysis.")

# Save the empty gene data for documentation
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Empty gene expression data saved to {out_gene_data_file}")

# 2. Extract clinical features for documentation purposes
if trait_row is not None:
    try:
        # Extract clinical features using the geo_select_clinical_features function
        clinical_features = geo_select_clinical_features(
            clinical_data, 
            trait=trait, 
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical features: {e}")
        clinical_features = pd.DataFrame()

# 3. Mark this dataset as unusable since we have empty gene data
note = "Dataset cannot be used for analysis due to empty gene expression data. The gene mapping and normalization process resulted in 0 rows of data."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=False,  # Mark as False since we have 0 rows
    is_trait_available=(trait_row is not None),
    is_biased=True,  # Mark as biased since we can't properly analyze
    df=pd.DataFrame(),  # Use empty DataFrame
    note=note
)

print(f"Dataset {cohort} is not usable for {trait} analysis due to empty gene expression data.")

Gene data shape: (0, 47)
Gene data is empty (0 rows). Cannot proceed with proper analysis.
Empty gene expression data saved to ../../output/preprocess/Head_and_Neck_Cancer/gene_data/GSE151181.csv
Clinical features saved to ../../output/preprocess/Head_and_Neck_Cancer/clinical_data/GSE151181.csv
Abnormality detected in the cohort: GSE151181. Preprocessing failed.
Dataset GSE151181 is not usable for Head_and_Neck_Cancer analysis due to empty gene expression data.
